In [1]:
from __future__ import annotations

In [2]:
with open("inputs.txt","r") as fp:
    data = fp.readlines()

In [10]:
from dataclasses import dataclass, field

In [4]:
from enum import Enum

# PART 1
Commands are:
1. cd
2. ls

In [15]:
class NodeType(Enum):
    dir: str = "dir"
    file: str = "file"

In [66]:
@dataclass
class Node:
    name: str
    ntype: NodeType
    parent: Node
    childern: dict[str, Node] = field(default_factory = dict)
    _size: int = None

    @property
    def size(self):
        if self.ntype == NodeType.file:
            return self._size
        else:
            return sum([v.size for _, v in self.childern.items()])

In [67]:
class FileSystem:
    def __init__(self) -> None:
        self.root = Node("/", NodeType.dir, None)
        self.cwd = self.root

In [68]:
fs = FileSystem()
listing = False
for line in data:
    parts: list[str] = line.strip().split(" ")
    if parts[0] == "$":
        listing = False
        if parts[1] == "cd":
            if parts[2] == "..":  # Out one
                fs.cwd = fs.cwd.parent
            elif parts[2] == "/":  # Root
                fs.cwd = fs.root
            else:  # In one
                fs.cwd = fs.cwd.childern[parts[2]]
        elif parts[1] == "ls":
            listing = True
        else:
            raise Exception("INVALID COMMAND")
    else:
        if parts[0].isnumeric():
            fs.cwd.childern[parts[1]] = Node(
                name=parts[1],
                ntype=NodeType.file,
                parent=fs.cwd,
                _size=int(parts[0]),
            )
        else:
            fs.cwd.childern[parts[1]] = Node(
                name=parts[1],
                ntype=NodeType.dir,
                parent=fs.cwd,
            )


In [70]:
canidates = []
to_search: list[Node] = [fs.root]
while len(to_search) > 0:
    for _, child in to_search.pop(0).childern.items():
        if child.ntype == NodeType.dir:
            
            if child.size <= 100000:
                canidates.append(child)
            to_search.append(child)

for canidate in canidates:
    assert canidate.size <= 100000

In [57]:
total = 0
for canidate in canidates:
    assert canidate.size() <= 100000
    total += canidate.size()
total

1086293

# Part 2

In [ ]:
# Current: 70 000 000
# Needed: 30 000 000

In [79]:
disk = 70000000
needed = 30000000
used = fs.root.size
available = disk-used

In [95]:
delta = needed - available; delta

358913

In [97]:
canidates = []
to_search: list[Node] = [fs.root]
while len(to_search) > 0:
    for _, child in to_search.pop(0).childern.items():
        if child.ntype == NodeType.dir:
            
            if child.size >= delta:
                canidates.append(child)
            to_search.append(child)

In [99]:
possible = min(filter(lambda x: x.size >= delta, canidates), key=lambda x: x.size)

In [101]:
possible.size

366028